In [1]:
import requests

use_real_puzzle_data = False # else use demo data
separator = '\n' if True else ','

github_site_user = "https://raw.githubusercontent.com/stephenhumphrey/"
repository_branch = "advent-of-code/main/2021/12/18/"

names_files = [("magnitudes","demo-magnitudes"),
               ("demo","demo-summations"),
               ("puzzle","puzzle-data")]

data = {source:{"name":source,
                "url":f"{github_site_user}{repository_branch}{file}.txt"} 
        for source,file in names_files}

for name,source in data.items():
    print(f"{name}: {source['url']}")

magnitudes: https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/demo-magnitudes.txt
demo: https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/demo-summations.txt
puzzle: https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/puzzle-data.txt


In [2]:
infinity = 99999999999999999999999999

do_verbose = False
def throwaway_verbose(unused):
    return
verbose = print if do_verbose else throwaway_verbose

In [3]:
def verbose_list(plist_string,max_elements = 5):
    plist = eval(plist_string)
    if len(plist) <= max_elements:
        sub = f"{plist}"
    elif max_elements < 2:
        sub = f"[{plist[0]}, ...]"
    else:
        half_max = max_elements // 2
        sub = "["
        for v in plist[0:half_max]:
            sub += f"{v}, "
        sub += "... "
        for v in plist[-half_max:-1]:
            sub += f"{v}, "
        sub += f"{plist[-1]}]"
        
    print(f"len({plist_string})={len(plist)}, {plist_string}={sub}")

In [4]:
for source_name,source in data.items():
    raw_data = requests.get(source["url"]).text
    data_lines = [line for line in raw_data.split(separator) if len(line.strip()) > 0]
    source["raw_lines"] = data_lines
    
    if False:
        print(source_name)
        for line in data_lines:
            print(line)
    
    if source_name == "puzzle":
        source["operands_raw"] = [data_lines]
        source["expected_values"] = ["unknown"]
    else:
        right = []
        source["operands_raw"] = []
        for line in data_lines:
            line_data = [v.strip() for v in line.split('=') if len(v.strip())>1]
            left = [v.strip() for v in line_data[0].split('+')]
            source["operands_raw"].append(left)
            right.append(line_data[1])
        source["expected_values"] = right

In [5]:
def separate_pair(pair_string):
    assert(pair_string[0]=='[' and pair_string[-1]==']')
    
    depth = 0
    split_at = 0
    for i,c in enumerate(pair_string):
        if c == '[':
            depth += 1
        elif c == ']':
            depth -= 1
        elif c == ',' and depth == 1:
            split_at = i
            break
    return pair_string[1:split_at], pair_string[split_at + 1:-1]

def parse_pair(pair_string):
    
    if pair_string[0] != '[':
        return int(pair_string)
    
    assert(pair_string[0]=='[' and pair_string[-1]==']')

    left,right = separate_pair(pair_string)
    left_parsed = parse_pair(left)
    right_parsed = parse_pair(right)
    
    return (left_parsed,right_parsed)

def pair_magnitude(pair):
    assert(type(pair)==type((9,9)) and len(pair)==2)
    
    left,right = pair
    assert(type(left)==type(9) or type(left)==type((9,9)))
    assert(type(right)==type(9) or type(right)==type((9,9)))
    
    sum_left_right = 0
    if type(left) == type(9):
        sum_left_right += left * 3
    else:
        sum_left_right += pair_magnitude(left) * 3
    
    if type(right) == type(9):
        sum_left_right += right * 2
    else:
        sum_left_right += pair_magnitude(right) * 2

    return sum_left_right

In [6]:
for source_name in data.keys():
    source = data[source_name]
    expected_values = source["expected_values"]
    pairs = []
    source["operands"] = pairs
    for i,operands in enumerate(source["operands_raw"]):
        print(f"{source_name}{i} operands({len(operands)}) expect:{expected_values[i]}")
        operand_pairs = []
        for j,operand in enumerate(operands):
            pair = parse_pair( operand )
            operand_pairs.append(pair)
            magnitude = pair_magnitude( pair )
            print(f"  operand[{j}] = {operand} pair = {pair} magnitude = {magnitude}")
        
        print(f"  pairs = {operand_pairs}")
        pairs.append(operand_pairs)

magnitudes0 operands(1) expect:29
  operand[0] = [9,1] pair = (9, 1) magnitude = 29
  pairs = [(9, 1)]
magnitudes1 operands(1) expect:21
  operand[0] = [1,9] pair = (1, 9) magnitude = 21
  pairs = [(1, 9)]
magnitudes2 operands(1) expect:129
  operand[0] = [[9,1],[1,9]] pair = ((9, 1), (1, 9)) magnitude = 129
  pairs = [((9, 1), (1, 9))]
magnitudes3 operands(1) expect:143
  operand[0] = [[1,2],[[3,4],5]] pair = ((1, 2), ((3, 4), 5)) magnitude = 143
  pairs = [((1, 2), ((3, 4), 5))]
magnitudes4 operands(1) expect:1384
  operand[0] = [[[[0,7],4],[[7,8],[6,0]]],[8,1]] pair = ((((0, 7), 4), ((7, 8), (6, 0))), (8, 1)) magnitude = 1384
  pairs = [((((0, 7), 4), ((7, 8), (6, 0))), (8, 1))]
magnitudes5 operands(1) expect:445
  operand[0] = [[[[1,1],[2,2]],[3,3]],[4,4]] pair = ((((1, 1), (2, 2)), (3, 3)), (4, 4)) magnitude = 445
  pairs = [((((1, 1), (2, 2)), (3, 3)), (4, 4))]
magnitudes6 operands(1) expect:791
  operand[0] = [[[[3,0],[5,3]],[4,4]],[5,5]] pair = ((((3, 0), (5, 3)), (4, 4)), (5, 

In [7]:
def unreduced_sum_of_two_pairs(pair0, pair1):
    if pair0 is None:
        return pair1
    if pair1 is None:
        return pair0
    return (pair0, pair1)

In [8]:
notes = """
To reduce a snailfish number, you must repeatedly do the first action in this list that applies to the snailfish number:

If any pair is nested inside four pairs, the leftmost such pair explodes.
If any regular number is 10 or greater, the leftmost such regular number splits.
Once no action in the above list applies, the snailfish number is reduced.

During reduction, at most one action applies, after which the process returns to the top of the list of actions. For example, if split produces a pair that meets the explode criteria, that pair explodes before other splits occur.

To explode a pair, the pair's left value is added to the first regular number to the left of the exploding pair (if any), and the pair's right value is added to the first regular number to the right of the exploding pair (if any). Exploding pairs will always consist of two regular numbers. Then, the entire exploding pair is replaced with the regular number 0.
Here are some examples of a single explode action:

[[[[[9,8],1],2],3],4] becomes [[[[0,9],2],3],4] (the 9 has no regular number to its left, so it is not added to any regular number).
[7,[6,[5,[4,[3,2]]]]] becomes [7,[6,[5,[7,0]]]] (the 2 has no regular number to its right, and so it is not added to any regular number).
[[6,[5,[4,[3,2]]]],1] becomes [[6,[5,[7,0]]],3].
[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]] (the pair [3,2] is unaffected because the pair [7,3] is further to the left; [3,2] would explode on the next action).
[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[7,0]]]].
To split a regular number, replace it with a pair; the left element of the pair should be the regular number divided by two and rounded down, while the right element of the pair should be the regular number divided by two and rounded up. For example, 10 becomes [5,5], 11 becomes [5,6], 12 becomes [6,6], and so on.

Here is the process of finding the reduced result of [[[[4,3],4],4],[7,[[8,4],9]]] + [1,1]:

after addition: [[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
after explode:  [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
after explode:  [[[[0,7],4],[15,[0,13]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
after explode:  [[[[0,7],4],[[7,8],[6,0]]],[8,1]]
Once no reduce actions apply, the snailfish number that remains is the actual result of the addition operation: [[[[0,7],4],[[7,8],[6,0]]],[8,1]].
"""

In [9]:
def reduced_sum_of_pairs(pairs):
    assert(len(pairs) > 0)
    
    print(f"  called reduced_sum_of_pairs() with {pairs}")
    
    previous_sum = None
    for pair in pairs:
        unreduced_sum = unreduced_sum_of_two_pairs(previous_sum,pair)
        print(f"  unreduced_sum = {unreduced_sum}")
        while reduce( unreduced_sum ):
            pass
        previous_sum = unreduced_sum        
    
    return previous_sum

In [10]:
def find_left_of(exploding_pairs,parents):
    left_peer = None
    immediate_parent = None
    exploding_pair = exploding_pairs[0]
    parent = parents[0]
    left,right = parent
    
    return left_peer, immediate_parent

In [11]:
def explode(parents, exploding_pairs):
    print(f"  called explode() with parents = {parents} exploding_pair = {exploding_pairs}")
    
    parent = parents[0]
    pair = exploding_pairs[0]
    left,right = pair
    
    assert(False)
    
    return

In [12]:
def reduce(pairs, parent = None, depth = 0):
    
    print(f"  called reduce() with {pairs}")
    
    pair = pairs[0]
    left,right = pair
    reduced = False
    
    if parent is None:
        parent = pairs[0]
    
    while True:
        pairs[0] = (left,right)
    
        if type(left) == type(9) and type(right) == type(9):
            # reg number
            if depth >= 4:
                print(f"explode {pair}, depth = {depth}")
                explode([parent],pairs)
                reduced = True
                continue 
        else:
            # inner pairs
            if type(left) != type(9):
                if reduce([left], parent, depth + 1):
                    reduced = True
                    continue
                    
            if type(right) != type(9):
                if reduce([right], parent, depth + 1):
                    reduced = True
                    continue
        
        break # we didn't reduce on this pass through the loop
    
    return reduced

# [[[[[9,8],1],2],3],4] becomes [[[[0,9],2],3],4]
# [7,[6,[5,[4,[3,2]]]]] becomes [7,[6,[5,[7,0]]]]
# [[6,[5,[4,[3,2]]]],1] becomes [[6,[5,[7,0]]],3]
# [[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
# [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[7,0]]]]
test_reduce = {}
test_reduce["[[[[0,9],2],3],4]"] = (((((9,8),1),2),3),4)
test_reduce["[7,[6,[5,[7,0]]]]"] = (7,(6,(5,(4,(3,2)))))
test_reduce["[[6,[5,[7,0]]],3]"] = ((6,(5,(4,(3,2)))),1)
test_reduce["[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"] = ((3,(2,(1,(7,3)))),(6,(5,(4,(3,2)))))
test_reduce["[[3,[2,[8,0]]],[9,[5,[7,0]]]]"] = ((3,(2,(8,0))),(9,(5,(4,(3,2)))))
for test_expected,test_pair in test_reduce.items():
    print(test_pair)
    reduce([test_pair])
    print(test_pair)
    print(test_expected)

(((((9, 8), 1), 2), 3), 4)
  called reduce() with [(((((9, 8), 1), 2), 3), 4)]
  called reduce() with [((((9, 8), 1), 2), 3)]
  called reduce() with [(((9, 8), 1), 2)]
  called reduce() with [((9, 8), 1)]
  called reduce() with [(9, 8)]
explode (9, 8), depth = 4
  called explode() with parents = [(((((9, 8), 1), 2), 3), 4)] exploding_pair = [(9, 8)]


AssertionError: 

In [ ]:
assert(False)

In [ ]:
for source_name in ["demo"]:
    source = data[source_name]
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}) expect:{expected_values[i]}")        
        unreduced_sum = reduced_sum_of_pairs(operands)
        print(f"  reduced_sum_of_pairs = {unreduced_sum}")

assert(False)

In [ ]:
for source_name,source in data.items():
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}):{operands} expect:{expected_values[i]}")
assert(False)

In [ ]:
def reduce(pair, parent = None, depth = 0):
    
    print(f"  called reduce() with {pair}, depth = {depth}")
    
    left,right = pair[0],pair[1]
    reduced = False
    
    if parent is None:
        parent = pair
    
    while True:
        pair[0],pair[1] = left,right
    
        if isinstance(left,int) and isinstance(right,int):
            # reg number
            if depth >= 4:
                print(f"explode {pair}, depth = {depth}")
                explode(parent,pair)
                reduced = True
        else:
            # inner pairs
            if not isinstance(left,int):
                if reduce(left, parent, depth + 1):
                    reduced = True
                    
            if not reduced and not isinstance(right,int):
                if reduce(right, parent, depth + 1):
                    reduced = True
        
        break # we didn't reduce on this pass through the loop
    
    return reduced

# [[[[[9,8],1],2],3],4] becomes [[[[0,9],2],3],4]
# [7,[6,[5,[4,[3,2]]]]] becomes [7,[6,[5,[7,0]]]]
# [[6,[5,[4,[3,2]]]],1] becomes [[6,[5,[7,0]]],3]
# [[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
# [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[7,0]]]]
test_reduce = {}
test_reduce["[[[[0,9],2],3],4]"] = [[[[[9,8],1],2],3],4]
test_reduce["[7,[6,[5,[7,0]]]]"] = [7,[6,[5,[4,[3,2]]]]]
test_reduce["[[6,[5,[7,0]]],3]"] = [[6,[5,[4,[3,2]]]],1]
test_reduce["[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"] = [[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]
test_reduce["[[3,[2,[8,0]]],[9,[5,[7,0]]]]"] = [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
for test_expected,test_pair in test_reduce.items():
    print('=' * 60)
    print(test_pair)
    while reduce(test_pair):
        pass
    print(test_pair)
    print(test_expected)

[3, 2] is not [4, [3, 2]]
calling find_left_of() pair = [3, 2] parents = [4, [3, 2]]
[3, 2] is not 4
[3, 2] is [3, 2]
found = True
left neighbor = (4, [4, [3, 2]], True)
immed = [4, [3, 2]]
calling find_right_of() pair = [3, 2] parents = [[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]]
[3, 2] is not [3, [2, [8, 0]]]
[3, 2] is not [9, [5, [4, [3, 2]]]]
calling find_right_of() pair = [3, 2] parents = [9, [5, [4, [3, 2]]]]
[3, 2] is not 9
[3, 2] is not [5, [4, [3, 2]]]
calling find_right_of() pair = [3, 2] parents = [5, [4, [3, 2]]]
[3, 2] is not 5
[3, 2] is not [4, [3, 2]]
calling find_right_of() pair = [3, 2] parents = [4, [3, 2]]
[3, 2] is not 4
[3, 2] is [3, 2]
found = True
right neighbor = None
immed = [4, [3, 2]]
after explosion: [[3, [2, [8, 0]]], [9, [5, [7, 0]]]]
  called reduce() with [[3, [2, [8, 0]]], [9, [5, [7, 0]]]], depth = 0
  called reduce() with [3, [2, [8, 0]]], depth = 1
  called reduce() with [2, [8, 0]], depth = 2
  called reduce() with [8, 0], depth = 3
  called reduce() 

In [14]:
assert(False)

AssertionError: 

In [ ]:
for source_name in ["demo"]:
    source = data[source_name]
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}) expect:{expected_values[i]}")        
        unreduced_sum = reduced_sum_of_pairs(operands)
        print(f"  reduced_sum_of_pairs = {unreduced_sum}")

assert(False)

In [ ]:
for source_name,source in data.items():
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}):{operands} expect:{expected_values[i]}")
assert(False)

In [ ]:
isinstance(9,int),isinstance((9,9),tuple),isinstance([],list)